# Zadanie
",
"
",
"- To jest szablon zadania - wypełnij przed wysłaniem
",
"- Zadanie polega na opracowaniu bazy danych klucz wartość oraz konfiguracji środowiska (serwera redis) i przedstawieniu wykonanej bazy.
",
"
",
" 1. jakie zapytanie będzie przyśpieszane (pojedyncze rekordy)?   {ocena 3.0-3.5}
",
" 2. jakie zapytanie będzie przyśpieszane (kilka rekordów)?    {ocena 3.0-4.5}
",
" 3. jakie zapytanie będzie przyśpieszane (złożone zapytanie np.: z joinem)?    {ocena 3.0-5.0}"

"Opis:
",
"-  Wykonaj bazę klucz wartość przyśpieszającą komunikację z Twoją bazę danych. Waro skorzystać z przygotowanej wcześniej bazy danych. Zastanów sie które zapytania będą najczęściej wykonywane, albo które będą wymagały dłuższego czasu ich wykonania. Dla takich zapytań napisz w notatniku metody pozwalające na przechowanie wyników zapytań w bazie redis. Opracuj rodzaj kluczy wg. których będziesz identyfikować odpowiednie dane w bazie redis.
",
"
",
" - W szablonie zadania opisz przynajmniej 3 optymalizowane zapytania,
",
" - oprogramuj zadanie,
",
" - prześlij wypełniony notatnik do naszego kursu online,
",
" - przedstaw działające zadanie na zajęciach laboratoryjnych,
",
" 
",
"- Nazwa pliku notatnika: zad3_nazwisko_imie.ipynb "

## Opis rozwiązania (wypełnia student)
",
"
",
"1. Autor: Maciej Rudy
",
" - ...........................
",
"2. jakie zadanie będzie rozwiązywane?
",
" 1. optymalizacja wyciągania danych z tabeli Stock
",
" 2. wyciąganie danych z tabeli Employees - stanowisko
",
" 3. wyciągany sprzedaży dla fabryk

In [1]:
# niezbędne importy
import redis
import cx_Oracle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [3]:
# połączenie z bazą sql
connection = cx_Oracle.connect(user="s108261", password="s108261",
                               dsn="217.173.198.135:1521/tpdb")
cursor = connection.cursor()
cursor.execute("ALTER SESSION SET nls_date_format='YYYY-MM-DD'")

In [4]:
# połączenie z bazą redis
r = redis.Redis(host='localhost', port=6379, db=0)

In [5]:
# Entry values
factory = 'Hybrid Oak'
dateS = '2020-11-30'
dateT = '2023-11-30'

name = "Karol"
surname = "Strognell"

In [6]:
# Common function
def sendToCache(index, data):
    try:
        r.set(index, data)
        print("success")
        return 0
    except:
        print("failed")
        return 1

In [12]:
# zadanie A
def getOracleStock(factory, dateS, dateT):
    querry = f"select * from STOCK INNER JOIN Factories on STOCK.factory_ID = Factories.factory_ID " \
           + f"where name = '{factory}' and PRODUCTION_DATE < '{dateT}' and " \
           + f"PRODUCTION_DATE > '{dateS}'"
    data = pd.read_sql(querry,connection)
    data = pickle.dumps(data)
    #dump
    return data

def getStockData(factory, dateS, dateT):
    index = (factory+":"+dateS+"-"+dateT)
    data = r.get(index)
    if data:
        print("Taken from Cache")
        #dump
        data = pickle.loads(data)
        return data

    else:
        data = getOracleStock(factory, dateS, dateT)
        sendToCache(index, data)
        print("Taken from Ocarle DB, pushed to cache")
        #dump
        data = pickle.loads(data)
        return data

newData = getStockData(factory, dateS, dateT)
print(type(newData))
print(newData)

success
Taken from Ocarle DB, pushed to cache
<class 'pandas.core.frame.DataFrame'>
   SERIALNUMBER_ID       SERIALNUMBER PRODUCTION_DATE  FACTORY_ID  SKU_ID  \
0              673  WAUSGAFBXAN414309      2023-05-22          20      52   
1              185  5UXZV4C51BL480539      2021-01-09          20      40   
2              352  YV1612FH5E2535721      2023-07-04          20       3   
3              584  1G6AC5SXXF0454738      2021-01-07          20      83   

   S_MEASURE_ID  FACTORY_ID        NAME COUNTRY POSTAL_CODE       CITY  
0           619          20  Hybrid Oak  Poland      27-420  Bodzechów  
1           409          20  Hybrid Oak  Poland      27-420  Bodzechów  
2           330          20  Hybrid Oak  Poland      27-420  Bodzechów  
3           415          20  Hybrid Oak  Poland      27-420  Bodzechów  


In [ ]:
# zadanie B
def getOracleEmpPos(name, surname):
    querry = f"SELECT employees.name, employees.surname, employees.position, factories.name AS factoryName FROM EMPLOYEES" \
         f" INNER JOIN Factories on EMPLOYEES.factory_ID = Factories.factory_ID" \
         f" WHERE employees.name = '{name}' AND employees.surname = '{surname}'"
    data = pd.read_sql(querry,connection)
    data = pickle.dumps(data)
    return data

def getEmpPosData(name, surname):
    index = (name+":"+surname)
    data = r.get(index)
    if data:
        print("Taken from Cache")
        data = pickle.loads(data)
        return data

    else:
        data = getOracleEmpPos(name, surname)
        sendToCache(index, data)
        print("Taken from Ocarle DB, pushed to cache")
        data = pickle.loads(data)
        return data

newData = getEmpPosData(name,surname)
print(type(newData))
print(newData)

In [ ]:
# zadanie C
def getOracleSales(factory):
    querry = f"SELECT AVG(INCOME), SALES.YEAR, Factories.name AS FactoryName FROM SALES" \
            f" INNER JOIN Factories on SALES.factory_ID = Factories.factory_ID" \
            f" WHERE Factories.name = '{factory}'" \
            f" GROUP BY Factories.name, SALES.YEAR"
    data = pd.read_sql(querry,connection)
    data = pickle.dumps(data)
    return data

def getSalesData(factory):
    index = (factory)
    data = r.get(index)
    if data:
        print("Taken from Cache")
        data = pickle.loads(data)
        return data

    else:
        data = getOracleSales(factory)
        sendToCache(index, data)
        print("Taken from Ocarle DB, pushed to cache")
        data = pickle.loads(data)
        return data

newData = getSalesData(factory)
print(type(newData))
print(newData)

In [11]:
# czyszczenie Cache
r.flushdb()

True